In [ ]:
import cv2
import os
import mediapipe as mp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
import numpy as np
import warnings

In [16]:
mp_hands = mp.solutions.hands

def process_image(image_path):
  if not os.path.exists(image_path):
    print(f"File {image_path} does not exist.")
    return None
  image = cv2.imread(image_path)
  if image is None:
    print(f"Failed to load image {image_path}.")
    return None
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  with mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.8) as hands:
    results = hands.process(image_rgb)
    if results.multi_hand_landmarks:
      landmarks = []
      for hand_landmarks in results.multi_hand_landmarks:
        for landmark in hand_landmarks.landmark:
          landmarks.extend([landmark.x, landmark.y, landmark.z])
      return landmarks
    else:
      return None


image_path_rock = 'dataset/archive/z_rock/1DxbXT3M2qiMjCmC.png'
image_path_zot = 'dataset/zot/20240525_143947.png'

landmarks_rock = process_image(image_path_rock)
landmarks_zot = process_image(image_path_zot)

print('Rock Landmarks:', landmarks_rock)
print('Zot Landmarks:', landmarks_zot)


Rock Landmarks: [0.7200443148612976, 0.3620128035545349, -8.134971380968636e-07, 0.6552819609642029, 0.5717319250106812, -0.006670438684523106, 0.5323469042778015, 0.6537334322929382, -0.018970472738146782, 0.4111444354057312, 0.6631207466125488, -0.027825310826301575, 0.32830825448036194, 0.6770933866500854, -0.028714919462800026, 0.4669240713119507, 0.5779258012771606, -0.05749014392495155, 0.3207486867904663, 0.5520085096359253, -0.08018173277378082, 0.37771016359329224, 0.5659164786338806, -0.0781286358833313, 0.4390231966972351, 0.5723342895507812, -0.07006395608186722, 0.46291470527648926, 0.4274115562438965, -0.06350831687450409, 0.3378012180328369, 0.43368852138519287, -0.06657560169696808, 0.3858932852745056, 0.45910000801086426, -0.04703783243894577, 0.4385814070701599, 0.45854902267456055, -0.03518667444586754, 0.47045040130615234, 0.29817041754722595, -0.0676112174987793, 0.37271639704704285, 0.3230329751968384, -0.06473401188850403, 0.41221559047698975, 0.3450777232646942,

I0000 00:00:1716678733.417776 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678733.423866 11319839 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678733.430096 11319841 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1716678733.463189 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678733.467085 11319845 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678733.471653 11319848 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [17]:
data = []
labels = []

gestures = ['zot', 'no_zot']

for gesture in gestures:
  folder_path = f'dataset/{gesture}'
  for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    landmarks = process_image(image_path)
    if landmarks and len(landmarks) == 63:
      data.append(landmarks)
      labels.append(gesture)

df = pd.DataFrame(data)
df['label'] = labels
df.to_csv('hand_gestures.csv', index=False)

df = pd.read_csv('hand_gestures.csv')
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

print(f'Model accuracy: {model.score(X_test, y_test)}')

joblib.dump(model, 'gesture_model.pkl')


I0000 00:00:1716678739.736093 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678739.740049 11319899 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678739.744912 11319899 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1716678739.778456 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678739.782475 11319910 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678739.786817 11319910 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1716678739.819338 11282396

Failed to load image dataset/zot/.DS_Store.


I0000 00:00:1716678741.060828 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678741.064757 11320184 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678741.069657 11320190 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1716678741.102848 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678741.106524 11320195 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678741.110984 11320195 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1716678741.130984 11282396

Failed to load image dataset/no_zot/.DS_Store.


I0000 00:00:1716678744.119299 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678744.123737 11320865 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678744.127965 11320865 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1716678744.160808 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678744.164727 11320874 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678744.169022 11320874 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1716678744.202424 11282396

Model accuracy: 0.8214285714285714


I0000 00:00:1716678746.887566 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678746.891927 11321450 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678746.896176 11321450 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


['gesture_model.pkl']

In [19]:
warnings.filterwarnings("ignore", category=UserWarning, module="google.protobuf.symbol_database")
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.base")

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

model = joblib.load('gesture_model.pkl')

cv2.namedWindow('Hand Tracking')
cap = cv2.VideoCapture(0)

def predict_gesture(landmarks):
  if len(landmarks) == 63:
    landmarks = np.array(landmarks).reshape(1, -1)
    prediction = model.predict(landmarks)
    return prediction[0]
  return None

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        landmarks = []
        for landmark in hand_landmarks.landmark:
          landmarks.extend([landmark.x, landmark.y, landmark.z])
        gesture = predict_gesture(landmarks)
        if gesture:
          print(gesture)
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


I0000 00:00:1716678750.782966 11282396 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
W0000 00:00:1716678750.786950 11321506 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716678750.791215 11321506 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


no_zot
no_zot
zot
no_zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
no_zot
no_zot
no_zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
no_zot
no_zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
no_zot
no_zot
no_zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
no_zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zot
zo

-1